In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
# Initial imports.
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import sklearn as skl
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
#Read the csv data into a dataframe
df = pd.read_csv(Path('Covid_19_dataV8_Confirmed_cases.csv'))
df

City                Province_State Country_Region      Lat      Long  \
0        NaN                           NaN      Argentina -38.4161  -63.6167   
1        NaN  Australian Capital Territory      Australia -35.4735  149.0124   
2        NaN               New South Wales      Australia -33.8688  151.2093   
3        NaN            Northern Territory      Australia -12.4634  130.8456   
4        NaN                    Queensland      Australia -27.4698  153.0251   
...      ...                           ...            ...      ...       ...   
1610312  NaN                      Selangor       Malaysia   3.0738  101.5183   
1610313  NaN                    Terengganu       Malaysia   5.3117  103.1324   
1610314  NaN             W.P. Kuala Lumpur       Malaysia   3.1390  101.6869   
1610315  NaN                   W.P. Labuan       Malaysia   5.2831  115.2308   
1610316  NaN                W.P. Putrajaya       Malaysia   2.9264  101.6964   

         Confirmed_Cases  Cases_Per_Day Last_Update  people_fully_vaccinated  \
0              7694506.0            NaN  2022-01-22                 34235912   
1                29240.0            NaN  2022-01-22                 20106005   
2               939615.0            NaN  2022-01-22                 20106005   
3                 8676.0            NaN  2022-01-22                 20106005   
4               298194.0            NaN  2022-01-22                 20106005   
...                  ...            ...         ...                      ...   
1610312         216483.0        25478.0  2021-06-15                  1488279   
1610313          10655.0          910.0  2021-06-15                  1488279   
1610314          67718.0         8116.0  2021-06-15                  1488279   
1610315           6154.0         2251.0  2021-06-15                  1488279   
1610316           2125.0          165.0  2021-06-15                  1488279   

                  Coordinates  
0        [-38.4161, -63.6167]  
1        [-35.4735, 149.0124]  
2        [-33.8688, 151.2093]  
3        [-12.4634, 130.8456]  
4        [-27.4698, 153.0251]  
...                       ...  
1610312    [3.0738, 101.5183]  
1610313    [5.3117, 103.1324]  
1610314     [3.139, 101.6869]  
1610315    [5.2831, 115.2308]  
1610316    [2.9264, 101.6964]  

[1610317 rows x 10 columns]

## Preparing the data set 

In [3]:
#Filter for the US
US_df = df.loc[df['Country_Region'] =='US']
US_df.head()

City Province_State Country_Region        Lat       Long  \
382  Autauga        Alabama             US  32.539527 -86.644082   
383  Baldwin        Alabama             US  30.727750 -87.722071   
384  Barbour        Alabama             US  31.868263 -85.387129   
385     Bibb        Alabama             US  32.996421 -87.125115   
386   Blount        Alabama             US  33.982109 -86.567906   

     Confirmed_Cases  Cases_Per_Day Last_Update  people_fully_vaccinated  \
382          13019.0            NaN  2022-01-22                210820617   
383          49168.0            NaN  2022-01-22                210820617   
384           4902.0            NaN  2022-01-22                210820617   
385           5663.0            NaN  2022-01-22                210820617   
386          13131.0            NaN  2022-01-22                210820617   

                           Coordinates  
382        [32.53952745, -86.64408227]  
383        [30.72774991, -87.72207058]  
384           [31.868263, -85.3871286]  
385  [32.99642064, -87.12511459999996]  
386        [33.98210918, -86.56790593]

In [4]:
Final_US_df = US_df.loc[US_df['Cases_Per_Day'] >=1]
                          
Final_US_df.head()

City Province_State Country_Region        Lat       Long  \
3590  Autauga        Alabama             US  32.539527 -86.644082   
3591  Baldwin        Alabama             US  30.727750 -87.722071   
3592  Barbour        Alabama             US  31.868263 -85.387129   
3593     Bibb        Alabama             US  32.996421 -87.125115   
3594   Blount        Alabama             US  33.982109 -86.567906   

      Confirmed_Cases  Cases_Per_Day Last_Update  people_fully_vaccinated  \
3590           3741.0         3697.0  2020-12-21                    11431   
3591          11827.0        11680.0  2020-12-21                    11431   
3592           1336.0         1304.0  2020-12-21                    11431   
3593           1613.0         1571.0  2020-12-21                    11431   
3594           4218.0         4164.0  2020-12-21                    11431   

                            Coordinates  
3590        [32.53952745, -86.64408227]  
3591        [30.72774991, -87.72207058]  
3592           [31.868263, -85.3871286]  
3593  [32.99642064, -87.12511459999996]  
3594        [33.98210918, -86.56790593]

In [5]:
len(Final_US_df)

272866

In [6]:
#Filter for NY(DATAFRAME TO BE MERGED WITH PREDICTATED CONFIRMED CASES)
New_df = Final_US_df.loc[Final_US_df['Province_State'] =='New York']
New_df.tail()

City Province_State Country_Region        Lat       Long  \
1575681   Washington       New York             US  43.311538 -73.430434   
1575682        Wayne       New York             US  43.154944 -77.029765   
1575683  Westchester       New York             US  41.162784 -73.757417   
1575684      Wyoming       New York             US  42.701451 -78.221996   
1575685        Yates       New York             US  42.635055 -77.103699   

         Confirmed_Cases  Cases_Per_Day Last_Update  people_fully_vaccinated  \
1575681           1006.0          989.0  2021-01-07                   637797   
1575682           2942.0         2899.0  2021-01-07                   637797   
1575683          74157.0        72460.0  2021-01-07                   637797   
1575684           1625.0         1598.0  2021-01-07                   637797   
1575685            608.0          595.0  2021-01-07                   637797   

                         Coordinates  
1575681  [43.31153794, -73.43043448]  
1575682  [43.15494365, -77.02976528]  
1575683  [41.16278376, -73.75741653]  
1575684  [42.70145109, -78.22199579]  
1575685  [42.63505484, -77.10369919]

In [7]:
NY_New_df = New_df.drop(['Coordinates'], axis=1)
NY_New_df

City Province_State Country_Region        Lat       Long  \
5417          Albany       New York             US  42.600603 -73.977239   
5418        Allegany       New York             US  42.257484 -78.027505   
5419           Bronx       New York             US  40.852093 -73.862828   
5420          Broome       New York             US  42.159032 -75.813261   
5421     Cattaraugus       New York             US  42.247782 -78.679231   
...              ...            ...            ...        ...        ...   
1575681   Washington       New York             US  43.311538 -73.430434   
1575682        Wayne       New York             US  43.154944 -77.029765   
1575683  Westchester       New York             US  41.162784 -73.757417   
1575684      Wyoming       New York             US  42.701451 -78.221996   
1575685        Yates       New York             US  42.635055 -77.103699   

         Confirmed_Cases  Cases_Per_Day Last_Update  people_fully_vaccinated  
5417              9174.0         8994.0  2020-12-21                    11431  
5418              1559.0         1491.0  2020-12-21                    11431  
5419             78599.0        73525.0  2020-12-21                    11431  
5420              7004.0         6818.0  2020-12-21                    11431  
5421              2037.0         2004.0  2020-12-21                    11431  
...                  ...            ...         ...                      ...  
1575681           1006.0          989.0  2021-01-07                   637797  
1575682           2942.0         2899.0  2021-01-07                   637797  
1575683          74157.0        72460.0  2021-01-07                   637797  
1575684           1625.0         1598.0  2021-01-07                   637797  
1575685            608.0          595.0  2021-01-07                   637797  

[5270 rows x 9 columns]

In [8]:
#Encoded the categorical data
encoded_US_df = pd.get_dummies(NY_New_df, columns=["City","Province_State","Country_Region","Last_Update", "Lat", "Long" ])
encoded_US_df.head(50)

Confirmed_Cases  Cases_Per_Day  people_fully_vaccinated  City_Albany  \
5417           9174.0         8994.0                    11431            1   
5418           1559.0         1491.0                    11431            0   
5419          78599.0        73525.0                    11431            0   
5420           7004.0         6818.0                    11431            0   
5421           2037.0         2004.0                    11431            0   
5422           2020.0         2006.0                    11431            0   
5423           2770.0         2757.0                    11431            0   
5424           4083.0         4000.0                    11431            0   
5425            972.0          957.0                    11431            0   
5426            795.0          790.0                    11431            0   
5427           1352.0         1293.0                    11431            0   
5428           1713.0         1694.0                    11431            0   
5429            581.0          574.0                    11431            0   
5430          10238.0        10023.0                    11431            0   
5431          36400.0        35343.0                    11431            0   
5432            439.0          423.0                    11431            0   
5433            576.0          567.0                    11431            0   
5434           1032.0         1003.0                    11431            0   
5435           2041.0         2011.0                    11431            0   
5436            987.0          969.0                    11431            0   
5437             82.0           81.0                    11431            0   
5438           1451.0         1433.0                    11431            0   
5439           1302.0         1296.0                    11431            0   
5440         111485.0       103871.0                    11431            0   
5441            655.0          644.0                    11431            0   
5442           1398.0         1377.0                    11431            0   
5443           1865.0         1825.0                    11431            0   
5444          27748.0        27327.0                    11431            0   
5445            948.0          919.0                    11431            0   
5446          76322.0        73997.0                    11431            0   
5447          55316.0        52048.0                    11431            0   
5448           6845.0         6689.0                    11431            0   
5449           9405.0         9194.0                    11431            0   
5450          17586.0        17261.0                    11431            0   
5451           2508.0         2462.0                    11431            0   
5452          21072.0        20500.0                    11431            0   
5453           1056.0          997.0                    11431            0   
5454           2782.0         2759.0                    11431            0   
5455           1005.0          997.0                    11431            0   
5456           4127.0         4061.0                    11431            0   
5457         113085.0       105626.0                    11431            0   
5458           3168.0         3098.0                    11431            0   
5459          31029.0        29807.0                    11431            0   
5460          25275.0        24711.0                    11431            0   
5461           4096.0         4068.0                    11431            0   
5462           4488.0         4417.0                    11431            0   
5463            418.0          415.0                    11431            0   
5464            441.0          433.0                    11431            0   
5465            586.0          567.0                    11431            0   
5466           1584.0         1552.0                    11431            0   

      City_Allegany  City_B

## Defining the features and target

In [9]:
#features set
X = encoded_US_df.copy()
X = encoded_US_df.drop('Confirmed_Cases', axis=1)
X.head

<bound method NDFrame.head of          Cases_Per_Day  people_fully_vaccinated  City_Albany  City_Allegany  \
5417            8994.0                    11431            1              0   
5418            1491.0                    11431            0              1   
5419           73525.0                    11431            0              0   
5420            6818.0                    11431            0              0   
5421            2004.0                    11431            0              0   
...                ...                      ...          ...            ...   
1575681          989.0                   637797            0              0   
1575682         2899.0                   637797            0              0   
1575683        72460.0                   637797            0              0   
1575684         1598.0                   637797            0              0   
1575685          595.0                   637797            0              0   

         City_Bronx  

In [10]:
X.shape

(5270, 275)

In [11]:
#target set.
y = encoded_US_df["Confirmed_Cases"]
y[:5]

5417     9174.0
5418     1559.0
5419    78599.0
5420     7004.0
5421     2037.0
Name: Confirmed_Cases, dtype: float64

## Spliting, fitting and predicting the results of the test set

In [12]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3952, 275) (1318, 275) (3952,) (1318,)


In [13]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 20, random_state = 42)
regressor = regressor.fit(X_train, y_train)

In [14]:
y_pred = regressor.predict(X_test)
y_pred

array([  2311.9 , 102758.1 ,   9733.15, ...,    678.3 , 127987.75,
         4458.85])

In [15]:
#print the realvalues vs. the predictated values
df = pd.DataFrame({'Real Values':y_test, 'Predicted Values':y_pred})
df

Real Values  Predicted Values
1064418       2342.0           2311.90
790559      101816.0         102758.10
1402858       9780.0           9733.15
1223311       1469.0           1468.10
1230242       4647.0           4631.05
...              ...               ...
449381       49671.0          49917.25
606097        3797.0           3795.90
1517784        675.0            678.30
282434      128917.0         127987.75
959668        4458.0           4458.85

[1318 rows x 2 columns]

In [16]:
#calculate the accuracy score
accuracy = regressor.score(X_train,y_train)
accuracy

0.9999696855873852

In [17]:
print("Training Accuracy = ", regressor.score(X_train, y_train))
print("Test Accuracy = ", regressor.score(X_test, y_test))

Training Accuracy =  0.9999696855873852
Test Accuracy =  0.999857236597642


In [18]:
# Making predictions using the testing data.
predictions = regressor.predict(X_test)
predictions

array([  2311.9 , 102758.1 ,   9733.15, ...,    678.3 , 127987.75,
         4458.85])

In [19]:
# Calculate feature importance in the Random Forest model.
importances = regressor.feature_importances_
importances

array([9.99472946e-01, 4.38211457e-04, 2.70678880e-09, 8.00433019e-10,
       2.98368075e-06, 1.52342681e-09, 1.31543615e-10, 4.04962288e-10,
       8.01965810e-10, 1.88644841e-10, 1.00183053e-10, 8.45262090e-10,
       8.15554303e-10, 1.99331836e-10, 6.41845509e-11, 4.27984740e-09,
       4.41213267e-08, 3.43198530e-11, 2.77913140e-10, 1.97300531e-10,
       1.31094288e-09, 1.49889700e-10, 5.72472252e-08, 3.79322122e-10,
       6.19890852e-10, 1.38686694e-06, 1.06579142e-10, 1.82518941e-10,
       3.80180119e-10, 9.06933317e-08, 8.31521555e-10, 1.81275705e-06,
       1.01740339e-06, 1.12231368e-09, 1.44908851e-09, 6.64983141e-09,
       1.47042172e-10, 1.12367489e-08, 1.28653426e-09, 2.00871893e-10,
       1.32842796e-10, 1.21363862e-09, 1.68251716e-06, 4.96597804e-10,
       9.55160387e-08, 1.62351649e-08, 2.34021073e-09, 7.31936490e-10,
       1.07760013e-10, 7.63959744e-12, 4.44042848e-10, 1.94290262e-10,
       3.48486377e-09, 2.20513714e-06, 2.16161526e-10, 1.17009151e-10,
      

In [20]:
#We can sort the features by their importance.
sorted(zip(regressor.feature_importances_, X.columns), reverse=True)

[(0.9994729458153099, 'Cases_Per_Day'),
 (0.0004382114573921991, 'people_fully_vaccinated'),
 (5.418197511372874e-06, 'Lat_40.88320119'),
 (4.2322333589289215e-06, 'Long_-73.94935552'),
 (4.052489725067331e-06, 'Lat_40.85209301'),
 (3.929962262054508e-06, 'Long_-72.8012172'),
 (3.750949918270618e-06, 'Long_-73.58941873'),
 (3.5177532409387253e-06, 'Long_-73.86282755'),
 (2.9836807531781334e-06, 'City_Bronx'),
 (2.821087800428009e-06, 'Lat_40.74066522'),
 (2.366174126374376e-06, 'Long_-73.81684712'),
 (2.205137137867503e-06, 'City_Suffolk'),
 (2.204317403442446e-06, 'Lat_40.6361825'),
 (1.8127570455508706e-06, 'City_Nassau'),
 (1.6825171603342955e-06, 'City_Queens'),
 (1.5986412332323689e-06, 'Last_Update_2021-02-17'),
 (1.5567470333299785e-06, 'City_Westchester'),
 (1.5080187436932733e-06, 'Long_-73.75741653'),
 (1.4638051441351321e-06, 'Lat_41.16278376'),
 (1.4037011743362137e-06, 'Last_Update_2021-01-22'),
 (1.3868669368144223e-06, 'City_Kings'),
 (1.3055886381622374e-06, 'Last_Updat

## Fitting and predicting the results of the whole data set

In [21]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 20,)
regressor = regressor.fit(X, y)

In [22]:
y_pred = regressor.predict(X)
y_pred

array([ 9160.75,  1553.75, 78406.4 , ..., 74599.5 ,  1629.45,   607.35])

In [23]:
#Print predictaed values for the full dataset
Predict_df = pd.DataFrame({'Real Values':y, 'Predicted Values':y_pred})
Predict_df

Real Values  Predicted Values
5417          9174.0           9160.75
5418          1559.0           1553.75
5419         78599.0          78406.40
5420          7004.0           6974.85
5421          2037.0           2042.15
...              ...               ...
1575681       1006.0           1008.50
1575682       2942.0           2941.90
1575683      74157.0          74599.50
1575684       1625.0           1629.45
1575685        608.0            607.35

[5270 rows x 2 columns]

## Merge the predictated values to the original dataframe

In [24]:
#Set index for the dataframe 
Predict_df.reset_index(inplace=True)
Predict_df

index  Real Values  Predicted Values
0        5417       9174.0           9160.75
1        5418       1559.0           1553.75
2        5419      78599.0          78406.40
3        5420       7004.0           6974.85
4        5421       2037.0           2042.15
...       ...          ...               ...
5265  1575681       1006.0           1008.50
5266  1575682       2942.0           2941.90
5267  1575683      74157.0          74599.50
5268  1575684       1625.0           1629.45
5269  1575685        608.0            607.35

[5270 rows x 3 columns]

In [25]:
Predict_df.rename(columns={'index':'Code'}, inplace= True)
Predict_df

Code  Real Values  Predicted Values
0        5417       9174.0           9160.75
1        5418       1559.0           1553.75
2        5419      78599.0          78406.40
3        5420       7004.0           6974.85
4        5421       2037.0           2042.15
...       ...          ...               ...
5265  1575681       1006.0           1008.50
5266  1575682       2942.0           2941.90
5267  1575683      74157.0          74599.50
5268  1575684       1625.0           1629.45
5269  1575685        608.0            607.35

[5270 rows x 3 columns]

In [29]:
New_df.reset_index(inplace=True)
New_df

index         City Province_State Country_Region        Lat  \
0        5417       Albany       New York             US  42.600603   
1        5418     Allegany       New York             US  42.257484   
2        5419        Bronx       New York             US  40.852093   
3        5420       Broome       New York             US  42.159032   
4        5421  Cattaraugus       New York             US  42.247782   
...       ...          ...            ...            ...        ...   
5265  1575681   Washington       New York             US  43.311538   
5266  1575682        Wayne       New York             US  43.154944   
5267  1575683  Westchester       New York             US  41.162784   
5268  1575684      Wyoming       New York             US  42.701451   
5269  1575685        Yates       New York             US  42.635055   

           Long  Confirmed_Cases  Cases_Per_Day Last_Update  \
0    -73.977239           9174.0         8994.0  2020-12-21   
1    -78.027505           1559.0         1491.0  2020-12-21   
2    -73.862828          78599.0        73525.0  2020-12-21   
3    -75.813261           7004.0         6818.0  2020-12-21   
4    -78.679231           2037.0         2004.0  2020-12-21   
...         ...              ...            ...         ...   
5265 -73.430434           1006.0          989.0  2021-01-07   
5266 -77.029765           2942.0         2899.0  2021-01-07   
5267 -73.757417          74157.0        72460.0  2021-01-07   
5268 -78.221996           1625.0         1598.0  2021-01-07   
5269 -77.103699            608.0          595.0  2021-01-07   

      people_fully_vaccinated                  Coordinates  
0                       11431  [42.60060306, -73.97723916]  
1                       11431  [42.25748406, -78.02750466]  
2                       11431  [40.85209301, -73.86282755]  
3                       11431  [42.15903158, -75.81326086]  
4                       11431  [42.24778241, -78.67923096]  
...                       ...                          ...  
5265                   637797  [43.31153794, -73.43043448]  
5266                   637797  [43.15494365, -77.02976528]  
5267                   637797  [41.16278376, -73.75741653]  
5268                   637797  [42.70145109, -78.22199579]  
5269                   637797  [42.63505484, -77.10369919]  

[5270 rows x 11 columns]

In [30]:
New_df.rename(columns={'index':'Code'}, inplace= True)
New_df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Code         City Province_State Country_Region        Lat  \
0        5417       Albany       New York             US  42.600603   
1        5418     Allegany       New York             US  42.257484   
2        5419        Bronx       New York             US  40.852093   
3        5420       Broome       New York             US  42.159032   
4        5421  Cattaraugus       New York             US  42.247782   
...       ...          ...            ...            ...        ...   
5265  1575681   Washington       New York             US  43.311538   
5266  1575682        Wayne       New York             US  43.154944   
5267  1575683  Westchester       New York             US  41.162784   
5268  1575684      Wyoming       New York             US  42.701451   
5269  1575685        Yates       New York             US  42.635055   

           Long  Confirmed_Cases  Cases_Per_Day Last_Update  \
0    -73.977239           9174.0         8994.0  2020-12-21   
1    -78.027505           1559.0         1491.0  2020-12-21   
2    -73.862828          78599.0        73525.0  2020-12-21   
3    -75.813261           7004.0         6818.0  2020-12-21   
4    -78.679231           2037.0         2004.0  2020-12-21   
...         ...              ...            ...         ...   
5265 -73.430434           1006.0          989.0  2021-01-07   
5266 -77.029765           2942.0         2899.0  2021-01-07   
5267 -73.757417          74157.0        72460.0  2021-01-07   
5268 -78.221996           1625.0         1598.0  2021-01-07   
5269 -77.103699            608.0          595.0  2021-01-07   

      people_fully_vaccinated                  Coordinates  
0                       11431  [42.60060306, -73.97723916]  
1                       11431  [42.25748406, -78.02750466]  
2                       11431  [40.85209301, -73.86282755]  
3                       11431  [42.15903158, -75.81326086]  
4                       11431  [42.24778241, -78.67923096]  
...                       ...                          ...  
5265                   637797  [43.31153794, -73.43043448]  
5266                   637797  [43.15494365, -77.02976528]  
5267                   637797  [41.16278376, -73.75741653]  
5268                   637797  [42.70145109, -78.22199579]  
5269                   637797  [42.63505484, -77.10369919]  

[5270 rows x 11 columns]

In [31]:
merged_df = New_df.merge(Predict_df, on=['Code'])
merged_df

Code         City Province_State Country_Region        Lat  \
0        5417       Albany       New York             US  42.600603   
1        5418     Allegany       New York             US  42.257484   
2        5419        Bronx       New York             US  40.852093   
3        5420       Broome       New York             US  42.159032   
4        5421  Cattaraugus       New York             US  42.247782   
...       ...          ...            ...            ...        ...   
5265  1575681   Washington       New York             US  43.311538   
5266  1575682        Wayne       New York             US  43.154944   
5267  1575683  Westchester       New York             US  41.162784   
5268  1575684      Wyoming       New York             US  42.701451   
5269  1575685        Yates       New York             US  42.635055   

           Long  Confirmed_Cases  Cases_Per_Day Last_Update  \
0    -73.977239           9174.0         8994.0  2020-12-21   
1    -78.027505           1559.0         1491.0  2020-12-21   
2    -73.862828          78599.0        73525.0  2020-12-21   
3    -75.813261           7004.0         6818.0  2020-12-21   
4    -78.679231           2037.0         2004.0  2020-12-21   
...         ...              ...            ...         ...   
5265 -73.430434           1006.0          989.0  2021-01-07   
5266 -77.029765           2942.0         2899.0  2021-01-07   
5267 -73.757417          74157.0        72460.0  2021-01-07   
5268 -78.221996           1625.0         1598.0  2021-01-07   
5269 -77.103699            608.0          595.0  2021-01-07   

      people_fully_vaccinated                  Coordinates  Real Values  \
0                       11431  [42.60060306, -73.97723916]       9174.0   
1                       11431  [42.25748406, -78.02750466]       1559.0   
2                       11431  [40.85209301, -73.86282755]      78599.0   
3                       11431  [42.15903158, -75.81326086]       7004.0   
4                       11431  [42.24778241, -78.67923096]       2037.0   
...                       ...                          ...          ...   
5265                   637797  [43.31153794, -73.43043448]       1006.0   
5266                   637797  [43.15494365, -77.02976528]       2942.0   
5267                   637797  [41.16278376, -73.75741653]      74157.0   
5268                   637797  [42.70145109, -78.22199579]       1625.0   
5269                   637797  [42.63505484, -77.10369919]        608.0   

      Predicted Values  
0              9160.75  
1              1553.75  
2             78406.40  
3              6974.85  
4              2042.15  
...                ...  
5265           1008.50  
5266           2941.90  
5267          74599.50  
5268           1629.45  
5269            607.35  

[5270 rows x 13 columns]